# POP MOC(sig2) for 0.1-degree
**Input Data:** Monthly POP output timeseries files  
**Output Data:** Monthly mean AMOC sigma 2 timeseries  
**Description:** Computes MOC(sig2) offline from POP history files using simple xhistogram binning.  
**Date:** February 2023  
**Creator:** Steve Yeager (https://github.com/sgyeager/POP_MOC/blob/main/notebooks/pop_MOCsig2_0.1deg.ipynb)  
**Updated:** Teagan King, February 2023 

In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import dask
from xhistogram.xarray import histogram
import pop_tools
import matplotlib.pyplot as plt
%matplotlib inline

from MOCutils import popmoc
import pop_tools

In [2]:
import dask
from dask.distributed import wait
dask.__version__

'2022.7.0'

### Start Dask Cluster

In [3]:
# Close out Dask Cluster and release workers:
cluster.close()
client.close()

NameError: name 'cluster' is not defined

In [4]:
# TODO: optimize dask resources

def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=1,
        memory='50GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=50GB',
        account='NCGD0011',
        walltime='03:00:00',
        interface='ib0',)

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
    })
    client = Client(cluster)
    return cluster, client

cluster, client = get_ClusterClient()
cluster.scale(35) 

/glade/u/home/tking/.conda/envs/ipogs/lib/python3.10/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/u/home/tking/.conda/envs/ipogs/lib/python3.10/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45956 instead
  warnings.warn(
/glade/u/home/tking/.conda/envs/ipogs/lib/python3.10/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now

In [5]:
cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/45956/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.48:46268,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/45956/status,Total threads: 0
Started: Just now,Total memory: 0 B


### 1. Read in OGCM history file & MOC template file

In [6]:
def time_set_midmonth(ds, time_name, deep=False):
    """
    Return copy of ds with values of ds[time_name] replaced with mid-month
    values (day=15) rather than end-month values.
    """
    #ds_out = ds.copy(deep)
    year = ds[time_name].dt.year
    month = ds[time_name].dt.month
    year = xr.where(month==1,year-1,year)
    month = xr.where(month==1,12,month-1)
    nmonths = len(month)
    newtime = [cftime.DatetimeNoLeap(year[i], month[i], 15) for i in range(nmonths)]
    ds[time_name] = newtime
    return ds

def preprocessor_uvel(ds0):
    keepvars = ['UVEL'] #,'VVEL','TEMP','SALT']
    d0 = ds0[keepvars]
    d0 = time_set_midmonth(d0,'time')
    #d0 = d0.groupby('time.year').mean('time').rename({'year':'time'})
    return d0

def preprocessor_vvel(ds0):
    keepvars = ['VVEL'] #,'VVEL','TEMP','SALT']
    d0 = ds0[keepvars]
    d0 = time_set_midmonth(d0,'time')
    #d0 = d0.groupby('time.year').mean('time').rename({'year':'time'})
    return d0

# def preprocessor_temp(ds0):
#     keepvars = ['TEMP'] #,'VVEL','TEMP','SALT']
#     d0 = ds0[keepvars]
#     d0 = time_set_midmonth(d0,'time')
#     #d0 = d0.groupby('time.year').mean('time').rename({'year':'time'})
#     return d0

def preprocessor_salt(ds0):
    keepvars = ['SALT'] #,'VVEL','TEMP','SALT']
    d0 = ds0[keepvars]
    d0 = time_set_midmonth(d0,'time')
    #d0 = d0.groupby('time.year').mean('time').rename({'year':'time'})
    return d0

In [12]:
%%time

# fdir = '/glade/campaign/cgd/oce/people/whokim/csm/g.e21.GRYF_0304.TL319_t13.003/ocn/hist/'
fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BHIST/HR/B.E.13.BHISTC5.ne120_t12.sehires38.003.sunway/ocn/proc/tseries/month_1/'
# TODO: this script takes *history* files as input

# TODO: loop through file years?
fin_uvel = fdir + '*UVEL*.nc'
ds_uvel = xr.open_mfdataset(fin_uvel,combine='by_coords',
                       preprocess=preprocessor_uvel,chunks={'time':1,'nlat':200},
                       coords="minimal",compat="override",data_vars="minimal",
                       parallel=True)

fin_vvel = fdir + '*VVEL*.nc'
ds_vvel = xr.open_mfdataset(fin_vvel,combine='by_coords',
                       preprocess=preprocessor_vvel,chunks={'time':1,'nlat':200},
                       coords="minimal",compat="override",data_vars="minimal",
                       parallel=True)

# fin_temp = fdir + '*TEMP*.nc'
# ds_temp = xr.open_mfdataset(fin_temp,combine='by_coords',
#                        preprocess=preprocessor_temp, chunks={'time':1,'nlat':200},
#                        coords="minimal",compat="override",data_vars="minimal",
#                        parallel=True)

# fin_salt = fdir + '*SALT*.nc'
# ds_salt = xr.open_mfdataset(fin_salt,combine='by_coords',
#                        preprocess=preprocessor_salt,chunks={'time':1,'nlat':200},
#                        coords="minimal",compat="override",data_vars="minimal",
#                        parallel=True)

ds_grid = pop_tools.get_grid('POP_tx0.1v3')

fmoc = '/glade/u/home/yeager/analysis/python/POP_MOC/moc_template.nc'
ds_moctemp = xr.open_dataset(fmoc)

CPU times: user 5.37 s, sys: 676 ms, total: 6.04 s
Wall time: 8.56 s


In [ ]:
%%time
dz = ds_grid['dz'].persist() / 100.
kmt = ds_grid['KMT'].fillna(0).persist() 
dzt,dzu = popmoc.tx0p1v3_dztdzu(dz,kmt)

### 2. Compute sigma-2 field from POP model output

In [9]:
# Get model T & S
salt = ds_salt['SALT']
temp = ds_temp['TEMP']

NameError: name 'ds_salt' is not defined

In [ ]:
%%time
refz = 2000
refdep = xr.full_like(ds_uvel['z_t'],refz).rename('REFDEP')

# Sigma2 on model TLAT, TLONG
sigma2_T = pop_tools.eos(salt=salt,temp=temp,depth=refdep) - 1000
sigma2_T = sigma2_T.assign_attrs({'long_name':'Sigma referenced to {}m'.format(refz),'units':'kg/m^3'})

In [ ]:
sigma2_T

### 3. Define target sigma-2 vertical grid
- Use a predefined target grid, or create your own!

In [ ]:
# Use predefined 86-layer sigma2 grid:
sigma_mid,sigma_edge = popmoc.sigma2_grid_86L()

In [ ]:
sigma_mid

In [ ]:
sigma_edge

### 4. Compute Isopycnal Layer Thickness (Can skip if not needed)

In [ ]:
%%time
# Here, test histogram by counting cells in each density bin. Vertical sum should be same as KMT.
iso_count = histogram(sigma2_T, bins=[sigma_edge.values],dim=['z_t'],density=False)
iso_count = iso_count.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

kmtdiff = iso_count.sum('sigma') - ds_grid['KMT']
print("Max difference from true KMT = {}".format(abs(kmtdiff).max().values))

In [ ]:
%%time
# Use histogram to compute layer thickness. Vertical sum should be same as HT.
iso_thick = histogram(sigma2_T, bins=[sigma_edge.values], weights=dzt,dim=['z_t'],density=False)
iso_thick = iso_thick.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
iso_thick = iso_thick.rename('iso_thick').assign_attrs({'units':'m','long_name':'Isopycnal Layer Thickness'}).rename({'sigma':'sigma_mid'})
iso_thick = iso_thick.transpose('time','sigma_mid','nlat','nlon')

htdiff = iso_thick.sum('sigma_mid') - (ds_grid['HT']/100.).assign_attrs({'units':'m'})
print("Max difference from true HT = {}m".format(abs(htdiff).max().values))

### 5. Compute Isopycnal Layer Depth (Can skip if not needed)

In [ ]:
# Cumulative sum of layer thickness yields depth of layer edges:
iso_depth = iso_thick.cumsum('sigma_mid').rename('iso_depth').rename({'sigma_mid':'sigma_bot'}).assign_attrs({'units':'m','long_name':'Isopycnal Layer Depth'})
sigma_bot = sigma_edge.isel(sigma=slice(1,None)).rename({'sigma':'sigma_bot'}).assign_attrs({'long_name':'Sigma2 at bottom of layer'})
iso_depth['sigma_bot'] = sigma_bot
iso_depth = iso_depth.transpose('time','sigma_bot','nlat','nlon')

In [ ]:
iso_depth

In [ ]:
iso_depth.isel(time=0,sigma_bot=84).plot(size=6,vmax=5500)

In [ ]:
%%time
# Isopycnal depth of bottom-most layer should be same as HT.
htdiff =  iso_depth.isel(sigma_bot=-1) - (ds_grid['HT']/100.).assign_attrs({'units':'m'})
print("Max difference from true HT = {}m".format(abs(htdiff).max().values))

### 6. Compute Isopycnal Layer Horizontal Volume Flux

In [ ]:
## Grid Metrics
dxu = ds_grid['DXU']
dyu = ds_grid['DYU']
dxt = ds_grid['DXT']
dyt = ds_grid['DYT']

In [ ]:
u_e = ds_uvel['UVEL']
u_e = u_e.where(u_e<1.e30,0)
v_e = ds_vvel['VVEL']
v_e = v_e.where(v_e<1.e30,0)

In [ ]:
%%time
# Grid-oriented Volume FLuxes:
u_e = (u_e*dyu*dzu/1.e4).assign_attrs({'units':'m^3/s'})
v_e = (v_e*dxu*dzu/1.e4).assign_attrs({'units':'m^3/s'})

In [ ]:
%%time
# Convert u_e,v_e to C-grid fluxes
u = 0.5*(u_e+u_e.shift(nlat=1))
v = 0.5*(v_e+v_e.roll(nlon=1,roll_coords=False))

In [ ]:
%%time
# Volume fluxes in density-space. 
iso_uflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=u,dim=['z_t'],density=False)
iso_uflux = iso_uflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

iso_vflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=v,dim=['z_t'],density=False)
iso_vflux = iso_vflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

In [ ]:
%%time
iso_uflux = iso_uflux.persist()
daskout = wait(iso_uflux)

In [ ]:
%%time
iso_vflux = iso_vflux.persist()
daskout = wait(iso_vflux)

In [ ]:
%%time
# Vertical sum in density-space should reproduce vertical sum in depth-space
ufluxdiff = iso_uflux.isel(time=0).sum('sigma') - u.isel(time=0).sum('z_t')
print("Max difference from true Uflux = {}".format(abs(ufluxdiff).max().values))
vfluxdiff = iso_vflux.isel(time=0).sum('sigma') - v.isel(time=0).sum('z_t')
print("Max difference from true Vflux = {}".format(abs(vfluxdiff).max().values))

### 7. Compute Vertical Volume Flux from horizontal flux convergence

In [ ]:
%%time
wflux = popmoc.wflux(iso_uflux,iso_vflux,'sigma',sigma_edge,grid='C')
wflux = wflux.assign_coords({'TLAT':ds_uvel['TLAT'],'TLONG':ds_uvel['TLONG']}).drop(['ULAT','ULONG'])

In [ ]:
%%time
wflux = wflux.persist()
daskout = wait(wflux)

### 8. Define MOC region masks

In [ ]:
## Define the MOC region mask:
rmask = ds_grid.REGION_MASK.drop(['ULONG','ULAT'])
rmaskglob = xr.where((rmask>0),1,0)
rmaskatl = xr.where((rmask>=6) & (rmask<=11),1,0)
rmaskmoc = xr.concat([rmaskglob,rmaskatl],dim=ds_moctemp.transport_regions)

In [ ]:
rmaskmoc.plot(levels=[0,1,2,3],col='transport_reg',size=5);

### 9. Compute MOC

In [ ]:
%%time
MOC = popmoc.compute_MOC(wflux,rmaskmoc,ds_moctemp.lat_aux_grid)
MOC = MOC.transpose('time','transport_reg','sigma','lat_aux_grid')

In [ ]:
%%time
MOC = MOC.load()
#daskout = wait(MOC)

### 10. Add Southern Boundary Fluxes for Atlantic Region

In [ ]:
# determine j=index of Atlantic region southern boundary
tmp = rmaskmoc.isel(transport_reg=1).sum('nlon')
atl_j = 0
j = 0
while (atl_j==0):
    if (tmp.isel(nlat=j).data>0):
        atl_j = j
    j += 1
atl_j = atl_j - 1
atl_j

In [ ]:
%%time
# add vflux at southern boundary of Atlantic domain
tmp = iso_vflux*(rmaskmoc.shift(nlat=-1))
tmp = tmp.isel(nlat=atl_j,transport_reg=1).sum('nlon')
moc_s = -tmp.sortby('sigma',ascending=False).cumsum('sigma').sortby('sigma',ascending=True)/1.e6
moc_s['sigma'] = sigma_edge.isel(sigma=slice(0,-1))
MOC[{'transport_reg':1}] = MOC[{'transport_reg':1}] + moc_s

In [ ]:
MOC.isel(time=0).isel(transport_reg=0).plot(ylim=[40,28])

In [ ]:
MOC.isel(time=0).isel(transport_reg=1).plot(ylim=[40,28])

### 11. Save to netcdf

In [ ]:
MOCann = MOC.groupby('time.year').mean('time').rename({'year':'time'})
dsout = MOCann.to_dataset()

In [ ]:
fout = '/glade/scratch/tking/MOCsig2.nc'
dsout.to_netcdf(fout,unlimited_dims='time')